In [ ]:
import numpy as np

from scipy.stats import norm, gamma
from scipy.optimize import differential_evolution, minimize

import matplotlib.pyplot as plt
plt.style.use("/Users/jlazar/Downloads/paper.mplstyle")

In [ ]:
a1true = 50
a2true = 35
loc1true = 2 
loc2true = 5 
scale1true = 0.5
scale2true = 1.5

scale = 5

times0 = np.append(
    np.random.normal(loc=loc1true, scale = scale1true, size=a1true),
    np.random.normal(loc=loc2true, scale = scale2true, size=a2true),
)
times1 = np.append(
    np.random.normal(loc=loc1true, scale = scale1true, size=a1true * scale),
    np.random.normal(loc=loc2true, scale = scale2true, size=a2true * scale),
)

In [ ]:
def cumulative_value(times, t0):
    return (times < t0).sum()

## Compare the timing distributions

Let make sure they look roughly the same

In [ ]:
ts = np.linspace(times0.min(), times0.max(), 100)
plt.plot(ts, [scale * cumulative_value(times0, t) for t in ts])
plt.plot(ts, [cumulative_value(times1, t) for t in ts])
plt.show()

## Utility function for getting a scaled CDF of two $\Gamma$ distributions

In [ ]:
def gamma_cdf(p):
    a1, a2, theta1, theta2, k1, k2, t1, t2 = p
    return lambda t: a1 * gamma.cdf(t - t1, theta1, k1) + a2 * gamma.cdf(t - t2, theta2, k2) 

In [ ]:
def norm_cdf(p):
    a1, a2, loc1, loc2, scale1, scale2 = p
    return lambda t: a1 * norm.cdf(t, loc=loc1, scale=scale1) + a2 * norm.cdf(t, loc=loc2, scale=scale2)

In [ ]:
ptrue = (a1true, a2true, loc1true, loc2true, scale1true, scale2true)

plt.plot(ts, norm_cdf(ptrue)(ts))
plt.plot(ts, [cumulative_value(times0, t) for t in ts])
plt.xlim(ts.min(), ts.max())
plt.xlabel(r"$t~\left[\mathrm{A.U}\right]$")
plt.ylabel(r"$N_{\mathrm{phot.}}$")
plt.show()


gcdf = norm_cdf(ptrue)
f_ = lambda t: -np.abs(cumulative_value(times0, t) - gcdf(t))
plt.plot(ts, [-f_(t) / times0.sum() for t in ts])
plt.xlim(ts.min(), ts.max())
plt.xlabel(r"$t~\left[\mathrm{A.U}\right]$")
plt.ylabel(r"Relative error")
plt.show()

In [ ]:
def f_test(times, p):
    a1, loc1, loc2, scale1, scale2 = p
    a2 = len(times) - a1
    gcdf = norm_cdf((a1, a2, loc1, loc2, scale1, scale2))
    f_ = lambda t: -np.abs(cumulative_value(times, t) - gcdf(t))
    res = differential_evolution(
        f_, 
        bounds=[(times.min(), times.max())]
    )
    # Uncomment this to experiment with using minimize
    # res = minimize(
    #     f_,
    #     (times.min()+times.max()) / 2,
    #     bounds=[(times.min(), times.max())],
    #     method="L-BFGS-B"
    # )
    return res

In [ ]:
ts[np.argsort(np.gradient(np.gradient([cumulative_value(times0, t) for t in ts])))[:7]]

In [ ]:
# This line is cheating
guess_loc1, guess_loc2 = 1.93, 4.448

MAXITER = 100

resss = []

for _ in range(10):
    times0 = np.append(
        np.random.normal(loc=loc1true, scale = scale1true, size=a1true),
        np.random.normal(loc=loc2true, scale = scale2true, size=a2true),
    )

    ress = []
    x0 = [
        53,
        guess_loc1,
        guess_loc2,
        1.6,
        1.6
    ]
    niter = 0
    niter_per = 20
    while niter < MAXITER:
        print(f"{niter+niter_per} iterations")
        g_test = lambda x: -f_test(times0, x).fun / times0.sum()
        res = differential_evolution(
            g_test,
            [
                (40, 60),
                (1, 4),
                (3, 6),
                (0.3, 2),
                (0.3, 2),
            ],
            x0=x0,
            # disp=True,
            recombination=0.0,
            # seed=1112,
            maxiter=niter_per,
            polish=True
        )
        niter += niter_per
        ress.append(res)
        # print(res.x)
        x0 = res.x
        pfit = tuple(res.x)
        pfit = (pfit[0], len(times0) - pfit[0], pfit[1], pfit[2], pfit[3], pfit[4])
        print(pfit)
        print(ptrue)
        plt.plot(ts, norm_cdf(pfit)(ts))
        plt.plot(ts, [cumulative_value(times0, t) for t in ts])
        plt.show()
        gaustrue = lambda x: ptrue[0] * norm.pdf(x, loc=ptrue[2], scale=ptrue[4]) + ptrue[1] * norm.pdf(x, loc=ptrue[3], scale=ptrue[5])
        gausfit = lambda x: pfit[0] * norm.pdf(x, loc=pfit[2], scale=pfit[4]) + pfit[1] * norm.pdf(x, loc=pfit[3], scale=pfit[5])
        plt.plot(ts, [gaustrue(t) for t in ts], label="True")
        plt.plot(ts, [gausfit(t) for t in ts], label="Fit")
        plt.legend(loc=1, fontsize=12)
        plt.show()
    resss.append(ress)
    #     gcdf = gamma_cdf_test(pfit)
    #     f_ = lambda t: -np.abs(cumulative_value(times0, t) - gcdf(t))
    #     plt.plot(ts, [f_(t) for t in ts])

    #     plt.show()